In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from glob import glob
import yaml
import numpy as np
import xarray as xr
import warnings

import regrid_tools
import cesm_tools

In [3]:
dst_grid = regrid_tools.grid('f09', grid_file='data/rectilinear_f09_SCRIP_20180107.nc')
dst_grid

grid: f09
dims: (192, 288)
file: data/rectilinear_f09_SCRIP_20180107.nc

In [4]:
src_grid = regrid_tools.grid('latlon', nx=360, ny=180, lon0=-180)
src_grid

exists: /glade/scratch/mclong/tmp/regridding/latlon_1.0x1.0_lon0=-180.0.nc


grid: latlon_latlon_1.0x1.0_lon0=-180.0
dims: (180, 360)
file: /glade/scratch/mclong/tmp/regridding/latlon_1.0x1.0_lon0=-180.0.nc

In [5]:
regrid_obj = regrid_tools.regridder(src_grid, dst_grid, method='conserve', clobber=True)
regrid_obj

generating: /glade/scratch/mclong/tmp/regridding/latlon_latlon_1.0x1.0_lon0=-180.0_to_f09_conserve_extrap-none.nc
 Starting weight generation with these inputs: 
   Source File: /glade/scratch/mclong/tmp/regridding/latlon_1.0x1.0_lon0=-180.0.nc
   Destination File: data/rectilinear_f09_SCRIP_20180107.nc
   Weight File: /glade/scratch/mclong/tmp/regridding/latlon_latlon_1.0x1.0_lon0=-180.0_to_f09_conserve_extrap-none.nc
   Source File is in SCRIP format
   Source Grid is a global grid
   Source Grid is a logically rectangular grid
   Use the center coordinates of the source grid to do the regrid
   Destination File is in SCRIP format
   Destination Grid is a global grid
   Destination Grid is a logically rectangular grid
   Use the center coordinates of the destination grid to do the regrid
   Regrid Method: conserve
   Pole option: NONE
   Ignore unmapped destination points
   Output weight file in NetCDF4 file format
   Line Type: greatcircle
   Norm Type: dstarea
   Extrap. Method: n

regridder latlon_1.0x1.0_lon0=-180.0.nc --> rectilinear_f09_SCRIP_20180107.nc

In [6]:
with xr.open_dataset('data/cam_grid.0.9x1.25.nc') as grid_coords:
    grid_coords['area'] = cesm_tools.get_area(grid_coords, 'atm') 
    grid_coords = grid_coords.drop([v for v in grid_coords.variables if v not in ['lat', 'lon', 'area']])
grid_coords

<xarray.Dataset>
Dimensions:  (lat: 192, lon: 288)
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Data variables:
    area     (lat, lon) float64 2.995e+07 2.995e+07 ... 2.995e+07 2.995e+07
Attributes:
    Conventions:       CF-1.0
    source:            CAM
    case:              cam6327_sd.apo_test_srf_emis.003
    logname:           mclong
    host:              cheyenne4
    initial_file:      /glade/work/mclong/cam-sd-cases/data/f.e20.FWAMIP.f09_...
    topography_file:   /glade/work/tilmes/inputdata/topo/fv_0.9x1.25_nc3000_N...
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    time_period_freq:  hour_6

In [7]:
clobber = True

flux_product_dir = "/glade/work/mclong/sno-analysis/sno-flux-products"
flux_product_camified = f"{flux_product_dir}-camified"
os.makedirs(flux_product_camified, exist_ok=True)

srf_emis_files_raw = {
    "apo_jena": f"{flux_product_dir}/SFAPO_OCN.carboscope.apo99X_v2021.1x1.19860101-20201231.nc",
    "co2_gridfed": f"{flux_product_dir}/SFCO2_FF.GCP-GridFED.v2021.3.1x1.19860101-20201231.nc",
    "co2_oco2mip": f"{flux_product_dir}/SFCO2_FF.OCO2-MIP.v2020.1.1x1.19860101-20201231.nc",
    "co2_cesm": f"{flux_product_dir}/SFCO2_OCN.cesm_fosi_smyle.1x1.19860101-20201231.nc",
    "co2_somffn": f"{flux_product_dir}/SFCO2_OCN.MPI-SOM-FFN.1x1.19860101-20201231.nc",
    "n2_cesm": f"{flux_product_dir}/SFN2_OCN.cesm_fosi_smyle.1x1.19860101-20201231.nc",
    "n2_era": f"{flux_product_dir}/SFN2_OCN.era5_shf.1x1.19860101-20201231.nc",
    "o2_gridfed": f"{flux_product_dir}/SFO2_FF.GCP-GridFED.v2021.3.1x1.19860101-20201231.nc",
    "o2_cesm": f"{flux_product_dir}/SFO2_OCN.cesm_fosi_smyle.1x1.19860101-20201231.nc",
    "o2_gk01r16": f"{flux_product_dir}/SFO2_OCN.gk2001_R2016.1x1.repeat_monclim.19860101-20201231.nc",
}

srf_emis_files_camified = {}
for v, path in srf_emis_files_raw.items():
    file_out = f"{flux_product_camified}/{os.path.basename(path)}".replace(
        ".1x1.", ".0.9x1.25."
    )

    flux_variable = os.path.basename(path).split(".")[0]
    constituent = flux_variable[2:].split("_")[0]
    constituent = "O2" if constituent == "APO" else constituent

    srf_emis_files_camified[v] = dict(
        flux_file=file_out,
        constituent=constituent,
        flux_variable=flux_variable,
    )
    if not os.path.exists(file_out) or clobber:

        with xr.open_dataset(path, use_cftime=True) as ds:
            ds = ds.load()

        # print(ds.time.values[0:5])
        # break
        flux_var = [
            v for v, da in ds.data_vars.items() if da.dims == ("time", "lat", "lon")
        ]
        assert len(flux_var) == 1
        flux_var = flux_var[0]
        assert ds[flux_var].attrs["units"] == "mol/m^2/s"

        with xr.set_options(keep_attrs=True):
            ds[flux_var] = (
                ds[flux_var] * cesm_tools.molecules_per_mol * cesm_tools.m2_per_cm2
            )
            ds[flux_var].attrs["units"] = "molecules/cm2/s"

        ds_f09 = xr.merge(
            (
                ds[[v for v in ds.data_vars if ds[v].dims[-2:] != ("lat", "lon")]],
                grid_coords,
            )
        )
        ds_f09[flux_var] = regrid_obj(ds[flux_var])

        # concatenate first and last values at end-points of time_bnds
        ds_t0 = ds_f09.isel(time=0)
        ds_t0['time'] = ds_f09[ds_f09.time.bounds].isel(time=0, d2=0)

        ds_tf = ds_f09.isel(time=-1)
        ds_tf['time'] = ds_f09[ds_f09.time.bounds].isel(time=-1, d2=1)

        dso = xr.concat((ds_t0, ds_f09, ds_tf), dim='time', data_vars='minimal')
        dso['time_components'] = cesm_tools.gen_time_components_variable(dso.time)

        year = dso.time_components[:, 0].astype(np.int32)
        month = dso.time_components[:, 1].astype(np.int32)
        day = dso.time_components[:, 2].astype(np.int32)

        dso["date"] = year * 10000 + month * 100 + day
        dso.date.attrs["units"] = "YYYYMMDD"
        dso.date.attrs["long_name"] = "Date"

        for vi in dso.data_vars:
            if len(dso[vi].dims) > 1 and 'time' in dso[vi].dims:
                non_time_dims = set(dso[vi].dims) - {'time'}
                ordered_dims = ['time'] + list(non_time_dims)
                dso[vi] = dso[vi].transpose(*ordered_dims)

        dso.time.attrs = ds_f09.time.attrs
        dso.time.encoding = ds_f09.time.encoding

        cesm_tools.to_netcdf_clean(dso, file_out)
        cesm_tools.ncks_fl_fmt64bit(file_out)


srf_emis_files_camified

------------------------------
Writing /glade/work/mclong/sno-analysis/sno-flux-products-camified/SFAPO_OCN.carboscope.apo99X_v2021.0.9x1.25.19860101-20201231.nc

netcdf SFAPO_OCN.carboscope.apo99X_v2021.0.9x1.25.19860101-20201231 {
dimensions:
	time = 12786 ;
	lat = 192 ;
	lon = 288 ;
	d2 = 2 ;
	n_time_components = 6 ;
variables:
	float SFAPO_OCN(time, lat, lon) ;
		SFAPO_OCN:_FillValue = 9.96921e+36f ;
		SFAPO_OCN:long_name = "Ocean-atmosphere apo flux" ;
		SFAPO_OCN:units = "molecules/cm2/s" ;
		SFAPO_OCN:cell_methods = "lon: lat: sum" ;
	float area(lat, lon) ;
		area:_FillValue = 9.96921e+36f ;
		area:units = "m2" ;
	int date(time) ;
		date:_FillValue = -2147483647 ;
		date:units = "YYYYMMDD" ;
		date:long_name = "Date" ;
	double lat(lat) ;
	double lon(lon) ;
	double time(time) ;
		time:bounds = "time_bnds" ;
		time:units = "days since 1986-01-01" ;
		time:calendar = "gregorian" ;
	double time_bnds(time, d2) ;
		time_bnds:_FillValue = 9.96920996838687e+36 ;
	int time_components(tim

{'apo_jena': {'flux_file': '/glade/work/mclong/sno-analysis/sno-flux-products-camified/SFAPO_OCN.carboscope.apo99X_v2021.0.9x1.25.19860101-20201231.nc',
  'constituent': 'O2',
  'flux_variable': 'SFAPO_OCN'},
 'co2_gridfed': {'flux_file': '/glade/work/mclong/sno-analysis/sno-flux-products-camified/SFCO2_FF.GCP-GridFED.v2021.3.0.9x1.25.19860101-20201231.nc',
  'constituent': 'CO2',
  'flux_variable': 'SFCO2_FF'},
 'co2_oco2mip': {'flux_file': '/glade/work/mclong/sno-analysis/sno-flux-products-camified/SFCO2_FF.OCO2-MIP.v2020.1.0.9x1.25.19860101-20201231.nc',
  'constituent': 'CO2',
  'flux_variable': 'SFCO2_FF'},
 'co2_cesm': {'flux_file': '/glade/work/mclong/sno-analysis/sno-flux-products-camified/SFCO2_OCN.cesm_fosi_smyle.0.9x1.25.19860101-20201231.nc',
  'constituent': 'CO2',
  'flux_variable': 'SFCO2_OCN'},
 'co2_somffn': {'flux_file': '/glade/work/mclong/sno-analysis/sno-flux-products-camified/SFCO2_OCN.MPI-SOM-FFN.0.9x1.25.19860101-20201231.nc',
  'constituent': 'CO2',
  'flux_var

In [8]:
with open('srf-emis-apo-forward.yml', 'w') as fid:
    yaml.dump(srf_emis_files_camified, fid)

In [9]:
ds

<xarray.Dataset>
Dimensions:          (time: 12784, lat: 180, lon: 360, d2: 2, n_time_components: 6)
Coordinates:
  * lat              (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * lon              (lon) float64 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * time             (time) object 1986-01-01 12:00:00 ... 2020-12-31 12:00:00
Dimensions without coordinates: d2, n_time_components
Data variables:
    SFO2_OCN         (time, lat, lon) float32 0.0 0.0 ... 1.832e+12 1.832e+12
    area             (lat, lon) float32 1.079e+08 1.079e+08 ... 1.079e+08
    time_bnds        (time, d2) object 1986-01-01 00:00:00 ... 2021-01-01 00:...
    time_components  (time, n_time_components) float32 1.986e+03 1.0 ... 0.0 0.0
Attributes:
    history:  Tue Feb 22 13:20:42 2022: ncks -O --fl_fmt=64bit /glade/scratch...
    NCO:      netCDF Operators version 4.9.5 (Homepage = http://nco.sf.net, C...